<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import slicer
import numpy as np

def load_image_series(image_folder):
    """ Load a series of 2D images from a folder into a 3D volume."""
    # Get sorted list of image files in the folder
    image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png') or f.endswith('.jpg')])

    # Load the first image to get dimensions
    first_image_node = slicer.util.loadVolume(image_files[0])
    image_shape = slicer.util.arrayFromVolume(first_image_node).shape
    slicer.mrmlScene.RemoveNode(first_image_node)

    # Create an empty 3D numpy array to hold the volume data
    volume_data = np.zeros((len(image_files), image_shape[0], image_shape[1]), dtype=np.int16)

    # Load each image and store it in the volume data array
    for i, image_file in enumerate(image_files):
        image_node = slicer.util.loadVolume(image_file)
        volume_data[i, :, :] = slicer.util.arrayFromVolume(image_node)
        slicer.mrmlScene.RemoveNode(image_node)

    return volume_data

def create_volume_node(volume_data):
    """ Create a Slicer volume node from a numpy array. """
    # Create a new volume node
    volume_node = slicer.vtkMRMLScalarVolumeNode()
    slicer.mrmlScene.AddNode(volume_node)

    # Convert the numpy array to VTK format and set it to the volume node
    slicer.util.updateVolumeFromArray(volume_node, volume_data)

    # Set necessary volume properties
    volume_node.SetSpacing(1, 1, 1)  # Modify as necessary
    volume_node.SetName("Volume from Images")

    return volume_node

def display_volume(volume_node):
    """ Display the volume using Slicer's volume rendering. """
    # Get the volume rendering logic
    volumeRenderingLogic = slicer.modules.volumerendering.logic()

    # Create a new volume rendering display node
    displayNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLVolumeRenderingDisplayNode')
    slicer.mrmlScene.AddNode(displayNode)

    # Set the volume node and the display node
    volumeRenderingLogic.UpdateDisplayNodeFromVolumeNode(displayNode, volume_node)

    # Adjust rendering properties if needed
    displayNode.SetVisibility(True)
    displayNode.SetAndObserveROINodeID(None)

    # Use a preset for better visualization
    displayNode.GetVolumePropertyNode().Copy(volumeRenderingLogic.GetPresetByName('CT-Chest-Contrast-Enhanced'))

    return displayNode

def main(image_folder):
    """ Main function to load images and create a 3D volume rendering. """
    volume_data = load_image_series(image_folder)
    volume_node = create_volume_node(volume_data)
    display_volume(volume_node)

# Replace with the path to your image folder
image_folder = 'path/to/your/image/folder'
main(image_folder)

In [ ]:

import os
import slicer
import numpy as np

def load_image_series(image_folder):
    """ Load a series of 2D images from a folder into a 3D volume."""
    # Get sorted list of image files in the folder
    image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png') or f.endswith('.jpg')])

    # Load the first image to get dimensions
    first_image_node = slicer.util.loadVolume(image_files[0])
    image_shape = slicer.util.arrayFromVolume(first_image_node).shape
    slicer.mrmlScene.RemoveNode(first_image_node)

    # Create an empty 3D numpy array to hold the volume data
    volume_data = np.zeros((len(image_files), image_shape[0], image_shape[1]), dtype=np.int16)

    # Load each image and store it in the volume data array
    for i, image_file in enumerate(image_files):
        image_node = slicer.util.loadVolume(image_file)
        volume_data[i, :, :] = slicer.util.arrayFromVolume(image_node)
        slicer.mrmlScene.RemoveNode(image_node)

    return volume_data

def create_volume_node(volume_data):
    """ Create a Slicer volume node from a numpy array. """
    # Create a new volume node
    volume_node = slicer.vtkMRMLScalarVolumeNode()
    slicer.mrmlScene.AddNode(volume_node)

    # Convert the numpy array to VTK format and set it to the volume node
    slicer.util.updateVolumeFromArray(volume_node, volume_data)

    # Set necessary volume properties
    volume_node.SetSpacing(1, 1, 1)  # Modify as necessary
    volume_node.SetName("Volume from Images")

    return volume_node

def display_volume(volume_node):
    """ Display the volume using Slicer's volume rendering. """
    # Get the volume rendering logic
    volumeRenderingLogic = slicer.modules.volumerendering.logic()

    # Create a new volume rendering display node
    displayNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLVolumeRenderingDisplayNode')
    slicer.mrmlScene.AddNode(displayNode)

    # Set the volume node and the display node
    volumeRenderingLogic.UpdateDisplayNodeFromVolumeNode(displayNode, volume_node)

    # Adjust rendering properties if needed
    displayNode.SetVisibility(True)
    displayNode.SetAndObserveROINodeID(None)

    # Use a preset for better visualization
    displayNode.GetVolumePropertyNode().Copy(volumeRenderingLogic.GetPresetByName('CT-Chest-Contrast-Enhanced'))

    return displayNode

def main(image_folder):
    """ Main function to load images and create a 3D volume rendering. """
    volume_data = load_image_series(image_folder)
    volume_node = create_volume_node(volume_data)
    display_volume(volume_node)

# Replace with the path to your image folder
image_folder = 'path/to/your/image/folder'
main(image_folder)

In [ ]:

#include <iostream>
#include <vector>
#include <thread>
#include <future>
#include <chrono>

// ビデオバイトのベクトルをリターンする関数
std::vector<unsigned char*> asyncFunction() {
    std::vector<unsigned char*> result;
    // ダミーデータの準備
    for (int i = 0; i < 10; ++i) {
        result.push_back(new unsigned char[i+1]);
    }
    // 処理時間をシミュレーション
    std::this_thread::sleep_for(std::chrono::seconds(2));
    return result;
}

int main() {
    // 非同期処理の開始
    std::future<std::vector<unsigned char*>> future = std::async(std::launch::async, asyncFunction);

    // 非同期処理の完了を待つ
    std::vector<unsigned char*> result = future.get();

    // リターン値を処理するためのループ
    for (auto ptr : result) {
        // ここでポインタを使った処理を行う
        std::cout << "Pointer address: " << static_cast<void*>(ptr) << std::endl;
        // メモリ解放
        delete[] ptr;
    }

    return 0;
}

In [ ]:

#include <iostream>
#include <vector>
#include <thread>
#include <future>
#include <chrono>
#include <tbb/tbb.h>
#include <immintrin.h>

// ビデオバイトのベクトルをリターンする関数
std::vector<unsigned char*> asyncFunction() {
    std::vector<unsigned char*> result(10);
    // TBBの並列forを使用してダミーデータの準備
    tbb::parallel_for(tbb::blocked_range<size_t>(0, result.size()), [&](const tbb::blocked_range<size_t>& r) {
        for (size_t i = r.begin(); i != r.end(); ++i) {
            result[i] = new unsigned char[i + 1];
        }
    });
    // 処理時間をシミュレーション
    std::this_thread::sleep_for(std::chrono::seconds(2));
    return result;
}

int main() {
    // 非同期処理の開始
    std::future<std::vector<unsigned char*>> future = std::async(std::launch::async, asyncFunction);

    // 非同期処理の完了を待つ
    std::vector<unsigned char*> result = future.get();

    // リターン値を並列に処理するためにTBBを使用
    tbb::parallel_for(tbb::blocked_range<size_t>(0, result.size()), [&](const tbb::blocked_range<size_t>& r) {
        for (size_t i = r.begin(); i != r.end(); ++i) {
            auto ptr = result[i];
            // SIMD命令を使用してデータを処理（ダミー処理としてゼロクリア）
            size_t size = _msize(ptr);
            __m256i zero = _mm256_setzero_si256();
            for (size_t j = 0; j < size; j += 32) {
                _mm256_storeu_si256(reinterpret_cast<__m256i*>(ptr + j), zero);
            }
            std::cout << "Pointer address: " << static_cast<void*>(ptr) << std::endl;
            // メモリ解放
            delete[] ptr;
        }
    });

    return 0;
}